In [38]:
# importing modules 
import numpy as np 
import pandas as pd 
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder 


In [39]:
#This fct almost always returns 1 or 0
def sigmoid(z): 
    return 1 / (1 + np.exp(-z))

In [40]:
data = pd.read_csv('cleaned_DrDoS_UDP.csv', header = 0)
print("Data Shape:", data.shape)
data.head()

Data Shape: (3136802, 85)


,Unnamed: 0.1,Unnamed: 0,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
0,0,101418.0,1721605.0,43443.0,192168501.0,6652.0,17.0,1.543668e+09,218395.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1,21564.0,1721605.0,54741.0,192168501.0,9712.0,17.0,1.543668e+09,108219.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,2,23389.0,1721605.0,56589.0,192168501.0,4680.0,17.0,1.543668e+09,104579.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,3,48872.0,1721605.0,40233.0,192168501.0,2644.0,17.0,1.543668e+09,110967.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,4,80354.0,1721605.0,33989.0,192168501.0,16901.0,17.0,1.543668e+09,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [41]:
#features
x_data = data.iloc[:,1:-1]
x_subset = x_data.iloc[0:10000]

#labels
y_data = data.iloc[:, -1:]
y_subset = y_data.iloc[0:10000]

x_subset.shape, y_subset.shape

((10000, 83), (10000, 1))

In [42]:
# Creating the One Hot Encoder 
oneHot = OneHotEncoder()
# # Encoding x_orig 
oneHot.fit(x_subset) 
x = oneHot.transform(x_subset).toarray() 
  
# Encoding y_orig 
oneHot.fit(y_subset) 
y = oneHot.transform(y_subset).toarray() 
  
alpha, epochs = 0.0035, 500
m, n = x.shape 
print('m =', m) 
print('n =', n) 
print('Learning Rate =', alpha) 
print('Number of Epochs =', epochs) 

m = 10000
n = 67370
Learning Rate = 0.0035
Number of Epochs = 500


In [43]:

tf.disable_v2_behavior()
# There are n columns in the feature matrix 
# after One Hot Encoding. 
X = tf.placeholder(tf.float32, [None, n]) #ISSUES WITH SIZE
  
# Since this is a binary classification problem, 
# Y can take only 2 values. 
Y = tf.placeholder(tf.float32, [n, 2]) 
  
# Trainable Variable Weights 
W = tf.Variable(tf.zeros([n, 2])) 
  
# Trainable Variable Bias 
b = tf.Variable(tf.zeros([2])) 

In [44]:
# Hypothesis 
Y_hat = tf.nn.sigmoid(tf.add(tf.matmul(X, W), b)) 

# Sigmoid Cross Entropy Cost Function 
cost = tf.nn.sigmoid_cross_entropy_with_logits( 
					logits = Y_hat, labels = Y) 

# Gradient Descent Optimizer 
optimizer = tf.train.GradientDescentOptimizer( 
		learning_rate = alpha).minimize(cost) 

# Global Variables Initializer 
init = tf.global_variables_initializer() 


In [45]:
# Starting the Tensorflow Session 
X = np.expand_dims(X,axis=-1)
with tf.Session() as sess: 
	print('hi')
	# Initializing the Variables 
	sess.run(init) 
	
	# Lists for storing the changing Cost and Accuracy in every Epoch 
	cost_history, accuracy_history = [], [] 
	
	# Iterating through all the epochs 
	for epoch in range(epochs): 
        
		cost_per_epoch = 0
		# Running the Optimizer 
		sess.run(optimizer, feed_dict = {X : x, Y : y}) 
		print('he')
		# Calculating cost on current Epoch 
		c = sess.run(cost, feed_dict = {X : x, Y : y}) 
		
		# Calculating accuracy on current Epoch 
		correct_prediction = tf.equal(tf.argmax(Y_hat, 1), 
										tf.argmax(Y, 1)) 
		accuracy = tf.reduce_mean(tf.cast(correct_prediction, 
												tf.float32)) 
		
		# Storing Cost and Accuracy to the history 
		cost_history.append(sum(sum(c))) 
		accuracy_history.append(accuracy.eval({X : x, Y : y}) * 100) 
		
		# Displaying result on current Epoch 
		if epoch % 100 == 0 and epoch != 0: 
			print("Epoch " + str(epoch) + " Cost: "
							+ str(cost_history[-1])) 
	
	Weight = sess.run(W) # Optimized Weight 
	Bias = sess.run(b) # Optimized Bias 
	
	# Final Accuracy 
	correct_prediction = tf.equal(tf.argmax(Y_hat, 1), 
									tf.argmax(Y, 1)) 
	accuracy = tf.reduce_mean(tf.cast(correct_prediction, 
											tf.float32)) 
	print("\nAccuracy:", accuracy_history[-1], "%") 


NotImplementedError: Cannot convert a symbolic tf.Tensor (Placeholder_12:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.